<a href="https://colab.research.google.com/github/jaegon-kim/python_study/blob/main/src/ai_essential_250317/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House Price Prediction
- **목표**
  - 이 워크샵은 주어진 데이터셋을 이용해 심층신경망 모델을 학습시켜 주택의 최종 판매 가격(SalePrice)을 예측하는 것이 최종 목표입니다.

- **데이터셋 정보**
  - 데이터셋은 총 79개의 설명 변수와 타겟 변수인 주택 가격(SalePrice)로 구성됩니다.
  - 설명 변수는 주택의 다양한 특성(예: 건축 연도, 면적, 위치, 방 개수 등)을 포함합니다.
  - 데이터는 판매 가격이 포함된 학습용 데이터인 `X`, `y` 와 판매 가격이 포함되지 않은 평가용 데이터인 `TEST`파일로 나뉘며, 각각 모델 학습 및 평가에 사용됩니다.
    - 평가용 데이터 `TEST`의 판매 가격(SalePrice)를 예측 후 리더보드로 제출하여 평가합니다.

- **문제 유형**
  - 이 워크샵은 회귀 문제로 연속형 변수를 예측하는 것이 목표입니다. 모델의 성능은 `Mean Absolute Error`로 측정됩니다.

## 1. 환경 설정

In [1]:
%%capture
!pip install JAEN -U

In [2]:
# 그대로 실행하세요.
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
username = "김재곤"
assert username, "username 변수에 값이 설정되지 않았습니다."

In [4]:
# 그대로 실행하세요.
from JAEN.competition import Competition
comp = Competition(
    username=username,
    course_name='AI Essential',
    course_round='0317(1)',
    competition_name='House Price Prediction'
)

## 2. 데이터 로드

In [5]:
from JAEN.datasets import load_house_price
X, y, TEST = load_house_price()
X.shape, y.shape, TEST.shape

(torch.Size([1460, 79]), torch.Size([1460, 1]), torch.Size([1459, 79]))

## 3. 제출 예시 코드

In [6]:
# TEST의 예측값 대입 (지금은 0으로 채워진 값 대입)
#comp.prediction =  torch.zeros(1459)
#comp.prediction

In [7]:
# 제출
#comp.submit()

## 4. 심층신경망 모델을 구성하고 학습하여 TEST를 예측해보세요.
- TEST의 예측 결과는 `comp.prediction`에 대입해주세요. **torch.tensor** 형태여야합니다.

In [8]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # 모델을 학습 모드로 설정

    running_loss = 0.0 # 미니 배치별 loss값을 누적할 변수

    for data, labels in train_loader: # 미니 배치 별 파라미터 업데이트 수행
        data, labels = data.to(device), labels.to(device) # 미니 배치별 데이터와 레이블 장치 할당

        # 순전파
        outputs = model(data)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 기울기 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 파라미터 업데이트
        optimizer.step()

        # 손실 누적
        running_loss += loss.item()

    # 현재 Epoch의 평균 손실 값 계산 및 반환
    # - len(train_loader): 평균 Loss
    return running_loss / len(train_loader)

In [9]:
def evaluate(model, test_loader, criterion, device):
    model.eval()  # 모델을 평가 모드로 설정

    running_loss = 0.0 # 미니 배치별 loss값을 누적할 변수

    with torch.no_grad():  # 평가 중에는 기울기 계산을 하지 않음
        for data, labels in test_loader: # 미니 배치 별 손실 계산
            data, labels = data.to(device), labels.to(device) # 미니 배치별 데이터와 레이블 장치 할당

            # 순전파
            outputs = model(data)

            # 손실 계산
            loss = criterion(outputs, labels)

            # 손실 누적
            running_loss += loss.item()


    # 현재 Epoch의 평균 손실 값 계산 및 반환
    return running_loss / len(test_loader)

In [10]:
# DataLoader 생성
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import KFold
#train_loader = DataLoader(TensorDataset(X, y), batch_size=32, shuffle=True)
dataset = TensorDataset(X, y)
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

In [11]:
'''
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(79, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # 10%의 드롭아웃 적용

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        #x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
'''
#'''
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc_1 = nn.Linear(79, 512)
        self.bn_1 = nn.BatchNorm1d(512)
        #self.fc0 = nn.Linear(79, 256)
        self.fc0 = nn.Linear(512, 256)
        self.bn0 = nn.BatchNorm1d(256)
        #self.fc1 = nn.Linear(79, 128)
        self.fc1 = nn.Linear(256, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # 10%의 드롭아웃 적용

    def forward(self, x):
        #x = self.relu(self.fc1(x))
        x = self.relu(self.bn_1(self.fc_1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn0(self.fc0(x)))
        x = self.dropout(x)
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
#'''

In [12]:
X.shape

torch.Size([1460, 79])

In [13]:
fold_models = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(dataset)):
    print(f'== Fold {fold+1} / {k_folds} ==')

    train_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)

    train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=16, shuffle=False)

    # Early Stopping을 위한 변수 설정
    best_loss = float('inf')
    best_epoch = 0
    patience = 3
    counter = 0

    train_losses = []
    test_losses = []

    # Fold 마다 Model 초기화
    model = DNN().to(device)

    # 손실함수 및 옵티마이저 설정
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)

    # 학습 횟수 만큼 반복
    for epoch in range(2000):

        # 모델 학습(학습데이터)
        train_loss = train(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)

        # 모델 평가 (평가데이터)
        test_loss = evaluate(model, test_loader, criterion, device)
        test_losses.append(test_loss)

        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1} Train Loss : {train_loss:,} Test Loss : {test_loss:,} in fold {fold}')

        # Early Stopping 조건 체크
        if test_loss < best_loss:
            best_loss = test_loss
            counter = 0
            best_epoch = epoch
            print(f'Save model Epoch {epoch+1} Train Loss : {train_loss:,} Test Loss : {test_loss:,}')
            torch.save(model.state_dict(), 'best_model.pt')
        else:
            counter += 1

        if counter >= patience:
            print(f'Early stopping trigerred at epoch {epoch+1} Best Loss: {best_loss:,} in epoch {best_epoch}')
            break

    model.load_state_dict(torch.load('best_model.pt'))

    fold_models.append(model)


== Fold 1 / 5 ==
Save model Epoch 1 Train Loss : 38,880,394,506.520546 Test Loss : 38,942,361,923.36842
Save model Epoch 2 Train Loss : 38,841,523,045.69863 Test Loss : 38,879,658,738.52631
Save model Epoch 3 Train Loss : 38,734,124,214.35616 Test Loss : 38,705,786,987.789474
Save model Epoch 4 Train Loss : 38,514,588,938.520546 Test Loss : 38,399,868,820.210526
Save model Epoch 5 Train Loss : 38,134,085,295.34247 Test Loss : 37,841,750,016.0
Save model Epoch 6 Train Loss : 37,555,337,272.10959 Test Loss : 37,235,211,964.63158
Save model Epoch 7 Train Loss : 36,755,295,105.753426 Test Loss : 36,552,837,443.36842
Save model Epoch 8 Train Loss : 35,735,474,049.753426 Test Loss : 35,301,953,428.210526
Save model Epoch 9 Train Loss : 34,440,560,808.328766 Test Loss : 34,163,452,227.36842
Save model Epoch 10 Train Loss : 32,850,486,903.232876 Test Loss : 32,369,901,568.0
Save model Epoch 11 Train Loss : 31,112,604,405.47945 Test Loss : 29,805,573,820.63158
Save model Epoch 12 Train Loss : 2

In [14]:
'''
m_cnt = 0
for m in fold_models:
    m_name = 'best_model' + str(m_cnt) + '.pt'
    torch.save(model.state_dict(), m_name)
    m_cnt += 1
'''

"\nm_cnt = 0\nfor m in fold_models:\n    m_name = 'best_model' + str(m_cnt) + '.pt'\n    torch.save(model.state_dict(), m_name)\n    m_cnt += 1\n"

In [15]:
# 학습된 모델의 TEST 예측

outputs = []
for model in fold_models:
    with torch.no_grad():
        output = model(TEST)
        outputs.append(output)
    avg_outputs = torch.mean(torch.stack(outputs), dim=0)


In [16]:
# comp.prediction에 TEST 예측 결과 대입
#comp.prediction = outputs
comp.prediction = avg_outputs
comp.prediction

tensor([[126773.5000],
        [190604.6875],
        [180696.6250],
        ...,
        [157227.8906],
        [ 61226.1758],
        [222542.9219]])

In [17]:
# 제출
comp.submit()

[House Price Prediction 평가 결과]
 AI Essential 0317(1) 과정 김재곤님의 점수는 16423.8359375 입니다.